# Ways to the Game of Life -- The Second Way

*Here we continue upon our dual-purpose journey: to learn Python by implementing
a version of Conway's Game of Life. Here we assume you have completed the
previous notebook to your satisfaction.*


**The Zen of Python**

>Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!

[Tim Peters -- PEP 20](https://www.python.org/dev/peps/pep-0020/)

Here we take inspiration from an implementation by Peter Norvig, which you should
examine for yourself [here](https://nbviewer.org/url/norvig.com/ipython/Life.ipynb).

I have placed a selection of the original code below, this code is in a markdown
cell and will not be processed by the interpreter.

```python
from collections import Counter

def next_generation(world):
    "The set of live cells in the next generation."
    possible_cells = counts = neighbor_counts(world)
    return {cell for cell in possible_cells
            if (counts[cell] == 3)
            or (counts[cell] == 2 and cell in world)}

def neighbor_counts(world):
    "A {cell: int} counter of the number of live neighbors for each cell that has neighbors."
    return Counter(nb for cell in world
                      for nb in neighbors(cell))

def neighbors(cell):
    "All 8 adjacent neighbors of cell."
    (x, y) = cell
    return [(x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1)]


world = {(3, 1), (1, 2), (1, 3), (2, 3)}
next_generation(world)
>>> {(1, 2), (1, 3), (2, 3)}  # Implied output.
```

Norvig has a unique style that I did not find particularly helpful when learning.
I recommend re-implementing this code yourself in a style you are comfortable with.
Before we get to coding, we should sketch out what we *think* is going on here.


First we trace how the functions call one another, and examine what *type* of
object is being input and output at each stage.

Most directly, we can implement the same `world` object and examine that absent the other code.

In [6]:
world = {(3, 1), (1, 2), (1, 3), (2, 3)}
world

{(1, 2), (1, 3), (2, 3), (3, 1)}

If you are not 100% sure of the type of object, check.

In [7]:
type(world)

set

That is not all there is to that object though, a `set`
is a container for other objects, so we should check to see
what type of object is inside. One thing you may think to try is
indexing from this `set` as you would a `list`.

In [10]:
world[0]

TypeError: 'set' object is not subscriptable

Sets are unordered, so there is no real meaning to my request for
the 0th indexed object. We can iterate through the set.

In [12]:
for cell in world:
    print(type(cell))
    break  # A nice way to test just one iteration of a loop.

<class 'tuple'>


Now that we have a notion of what *type* of objects are being used
for our model, we can examine the functions in more detail.

The first function called is `next_generation()`, which takes the
world object we just examined as its sole argument. The pattern
Norvig uses here is called a *generator*, and it is a method of
implementing loops.

That function immediately calls `neighbor_counts(world)`. Which
clearly takes the same world object as an input. Within that function
we see the call to `Counter()`, which you probably noticed imported
at the top of the code block.

Since this function is new to us, we are not sure what type of
object it returns. Though we can be certain that it can accept
a `set` object as input.

In [13]:
from collections import Counter

Remember to get in the practice of looking at the included documentation.

In [14]:
help(Counter)

Help on class Counter in module collections:

class Counter(builtins.dict)
 |  Counter(iterable=None, /, **kwds)
 |  
 |  Dict subclass for counting hashable items.  Sometimes called a bag
 |  or multiset.  Elements are stored as dictionary keys and their counts
 |  are stored as dictionary values.
 |  
 |  >>> c = Counter('abcdeabcdabcaba')  # count elements from a string
 |  
 |  >>> c.most_common(3)                # three most common elements
 |  [('a', 5), ('b', 4), ('c', 3)]
 |  >>> sorted(c)                       # list all unique elements
 |  ['a', 'b', 'c', 'd', 'e']
 |  >>> ''.join(sorted(c.elements()))   # list elements with repetitions
 |  'aaaaabbbbcccdde'
 |  >>> sum(c.values())                 # total of all counts
 |  15
 |  
 |  >>> c['a']                          # count of letter 'a'
 |  5
 |  >>> for elem in 'shazam':           # update counts from an iterable
 |  ...     c[elem] += 1                # by adding 1 to each element's count
 |  >>> c['a']                

In that is a key line that contains all the information we need:
> Elements are stored as dictionary keys and their counts
 |  are stored as dictionary values.

So, we know that a set goes in, and a dictionary object comes out.
A dictionary object is a builtin Python data type that stores key: value
pairs. In this case our keys will be the `tuple` objects that make up
the world `set` object.

Take it for a spin!

In [17]:
counted_world = Counter(world)
counted_world

Counter({(3, 1): 1, (2, 3): 1, (1, 2): 1, (1, 3): 1})

In [18]:
type(counted_world)

collections.Counter

In [19]:
counted_world[(3, 1)]

1

While that clarified what the `Counter()` call is doing, we notice
that Norvig's code counts the neighbors of all cells in the world,
rather than the cells themselves. So let's examine the `neighbors()`
function.

In [16]:
def neighbors(cell):
    (x, y) = cell
    return [(x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x-1, y+1), (x, y+1), (x+1, y+1)]

neighbors((3, 1))

[(2, 0), (3, 0), (4, 0), (2, 1), (4, 1), (2, 2), (3, 2), (4, 2)]